In [2]:
# !pip install wandb
# !pip install sklearn
# !pip install xgboost
# !pip install lightgbm

In [44]:
import os
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import wandb
from wandb.integration.xgboost import WandbCallback

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, precision_score
from sklearn.ensemble import RandomForestClassifier

from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier

import lightgbm as lgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)
warnings.filterwarnings('ignore')

In [98]:
class cfg:
    data_path = './data'
    df_path = os.path.join(data_path, 'acc_and_injury_df.pickle')

In [99]:
# wandb.init(
#     project="Hyundai AI"
# )

<font color=yellow>
1. data 불러오기
</font>

In [100]:
df_anj = pd.read_pickle(cfg.df_path)
df_anj.head()

,ALIGNMNT,BODYTYPE,CLIMATE,CURBWGT,DVLAT,DVLONG,IMPACTSP,PREEVENT,PREILOC,PREISTAB,PREMOVE,PROFILE,RELINTER,ROLLOVER,SURTYPE,TOWHITCH,VEHTYPE,MODELYR,VEHNO,HMC_CASE_ID,HMC_CASE_ID_VEHNO,MANCOLL,TIME,EVENTS,FIRE,GAD1,PDOF1,PDOF2,ORIGAVTW,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,OBJCONT,SEATPOS_NEW,AIS
0,1.0,1.0,999.0,161.0,12.0,20.0,998.0,1.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,2,1992.0,1.0,2005-2-1,2005-2-1-1,0.0,925.0,1.0,0.0,3.0,11.0,999.0,3.0,3.0,0.0,1.0,0.0,0.0,3.0,1.0,2
1,2.0,1.0,999.0,143.0,999.0,999.0,999.0,4.0,1.0,1.0,5.0,1.0,2.0,0.0,1.0,0.0,2,1989.0,2.0,2005-2-2,2005-2-2-2,4.0,1353.0,1.0,0.0,3.0,12.0,999.0,2.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0,1
2,1.0,1.0,999.0,161.0,999.0,999.0,999.0,4.0,1.0,999.0,1.0,1.0,0.0,0.0,1.0,0.0,2,2001.0,1.0,2005-2-3,2005-2-3-1,2.0,1111.0,2.0,0.0,2.0,12.0,999.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,2
3,2.0,1.0,999.0,120.0,0.0,20.0,998.0,3.0,1.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,2,2003.0,2.0,2005-2-4,2005-2-4-2,1.0,1248.0,2.0,0.0,1.0,6.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1
4,3.0,2.0,999.0,146.0,0.0,22.0,998.0,2.0,4.0,1.0,5.0,1.0,0.0,0.0,1.0,0.0,3,2004.0,1.0,2005-2-5,2005-2-5-1,0.0,1539.0,3.0,0.0,2.0,1.0,999.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1


In [101]:
for col in df_anj.columns:
    if df_anj[col].dtype == 'category':
        df_anj[col] = df_anj[col].astype('float')

<font color=yellow>
2. 전처리 및 데이터 확인
</font>

In [102]:
# 누락된 값 출력
df_anj[df_anj.isna().any(axis=1)]

,ALIGNMNT,BODYTYPE,CLIMATE,CURBWGT,DVLAT,DVLONG,IMPACTSP,PREEVENT,PREILOC,PREISTAB,PREMOVE,PROFILE,RELINTER,ROLLOVER,SURTYPE,TOWHITCH,VEHTYPE,MODELYR,VEHNO,HMC_CASE_ID,HMC_CASE_ID_VEHNO,MANCOLL,TIME,EVENTS,FIRE,GAD1,PDOF1,PDOF2,ORIGAVTW,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,OBJCONT,SEATPOS_NEW,AIS
51697,1.0,1.0,8.0,9999.0,999.0,999.0,999.0,999.0,1.0,1.0,1.0,999.0,1.0,0.0,1.0,0.0,999,2008.0,1.0,2019-21-15166,2019-21-15166-1,0.0,1505.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2


In [103]:
df_anj = df_anj.fillna(999)
df_anj[df_anj.isna().any(axis=1)]

,ALIGNMNT,BODYTYPE,CLIMATE,CURBWGT,DVLAT,DVLONG,IMPACTSP,PREEVENT,PREILOC,PREISTAB,PREMOVE,PROFILE,RELINTER,ROLLOVER,SURTYPE,TOWHITCH,VEHTYPE,MODELYR,VEHNO,HMC_CASE_ID,HMC_CASE_ID_VEHNO,MANCOLL,TIME,EVENTS,FIRE,GAD1,PDOF1,PDOF2,ORIGAVTW,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,OBJCONT,SEATPOS_NEW,AIS


In [104]:
# datatype 확인
df_anj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55146 entries, 0 to 55145
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ALIGNMNT           55146 non-null  float64
 1   BODYTYPE           55146 non-null  float64
 2   CLIMATE            55146 non-null  float64
 3   CURBWGT            55146 non-null  float64
 4   DVLAT              55146 non-null  float64
 5   DVLONG             55146 non-null  float64
 6   IMPACTSP           55146 non-null  float64
 7   PREEVENT           55146 non-null  float64
 8   PREILOC            55146 non-null  float64
 9   PREISTAB           55146 non-null  float64
 10  PREMOVE            55146 non-null  float64
 11  PROFILE            55146 non-null  float64
 12  RELINTER           55146 non-null  float64
 13  ROLLOVER           55146 non-null  float64
 14  SURTYPE            55146 non-null  float64
 15  TOWHITCH           55146 non-null  float64
 16  VEHTYPE            551

<font color=yellow>
3. 데이터 구성
</font>

In [106]:
X = df_anj.drop(columns=['AIS', 'HMC_CASE_ID', 'HMC_CASE_ID_VEHNO']) # , 'HMC_CASE_ID'
X

,ALIGNMNT,BODYTYPE,CLIMATE,CURBWGT,DVLAT,DVLONG,IMPACTSP,PREEVENT,PREILOC,PREISTAB,PREMOVE,PROFILE,RELINTER,ROLLOVER,SURTYPE,TOWHITCH,VEHTYPE,MODELYR,VEHNO,MANCOLL,TIME,EVENTS,FIRE,GAD1,PDOF1,PDOF2,ORIGAVTW,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,OBJCONT,SEATPOS_NEW
0,1.0,1.0,999.0,161.0,12.0,20.0,998.0,1.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,2,1992.0,1.0,0.0,925.0,1.0,0.0,3.0,11.0,999.0,3.0,3.0,0.0,1.0,0.0,0.0,3.0,1.0
1,2.0,1.0,999.0,143.0,999.0,999.0,999.0,4.0,1.0,1.0,5.0,1.0,2.0,0.0,1.0,0.0,2,1989.0,2.0,4.0,1353.0,1.0,0.0,3.0,12.0,999.0,2.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0
2,1.0,1.0,999.0,161.0,999.0,999.0,999.0,4.0,1.0,999.0,1.0,1.0,0.0,0.0,1.0,0.0,2,2001.0,1.0,2.0,1111.0,2.0,0.0,2.0,12.0,999.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0
3,2.0,1.0,999.0,120.0,0.0,20.0,998.0,3.0,1.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,2,2003.0,2.0,1.0,1248.0,2.0,0.0,1.0,6.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
4,3.0,2.0,999.0,146.0,0.0,22.0,998.0,2.0,4.0,1.0,5.0,1.0,0.0,0.0,1.0,0.0,3,2004.0,1.0,0.0,1539.0,3.0,0.0,2.0,1.0,999.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55141,3.0,8.0,9.0,0.0,999.0,999.0,999.0,6.0,4.0,2.0,5.0,1.0,0.0,0.0,1.0,0.0,999,2018.0,1.0,0.0,1638.0,2.0,0.0,2.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,0.0,3.0,1.0
55142,1.0,8.0,8.0,0.0,999.0,999.0,999.0,2.0,4.0,999.0,3.0,1.0,0.0,1.0,1.0,0.0,999,2002.0,1.0,0.0,2154.0,1.0,0.0,5.0,998.0,998.0,1.0,2.0,2.0,2.0,0.0,0.0,2.0,1.0
55143,1.0,8.0,8.0,544.0,999.0,999.0,999.0,3.0,1.0,1.0,1.0,999.0,0.0,1.0,1.0,1.0,999,2005.0,1.0,1.0,640.0,5.0,0.0,5.0,998.0,998.0,7.0,999.0,999.0,999.0,999.0,0.0,3.0,1.0
55144,1.0,8.0,8.0,544.0,999.0,999.0,999.0,3.0,1.0,1.0,1.0,999.0,0.0,1.0,1.0,1.0,999,2005.0,1.0,1.0,640.0,5.0,0.0,5.0,998.0,998.0,7.0,999.0,999.0,999.0,999.0,0.0,3.0,2.0


In [107]:
y = df_anj[['AIS']]
y

,AIS
0,2
1,1
2,2
3,1
4,1
...,...
55141,1
55142,1
55143,1
55144,2


In [108]:
X.shape, y.shape

((55146, 34), (55146, 1))

In [109]:
# train/test dataset 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape

((49631, 34), (5515, 34))

<font color=yellow>
4. Random forest 
</font>

In [110]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
cols = X_train.columns

X_train_rfc = scaler.fit_transform(X_train)
X_train_rfc = pd.DataFrame(X_train_rfc, columns=[cols]) 
X_test_rfc = scaler.transform(X_test)
X_test_rfc = pd.DataFrame(X_test_rfc, columns=[cols])

In [111]:
rfc = RandomForestClassifier(n_estimators=100,
                                random_state=42,
                                max_depth=100
                                )
rfc.fit(X_train_rfc, y_train)

RandomForestClassifier(max_depth=100, random_state=42)

In [112]:
y_pred_rfc = rfc.predict(X_test_rfc)
prc_rfc, rec_rfc = precision_score(y_pred_rfc, y_test, average='weighted'), recall_score(y_pred_rfc, y_test, average='weighted')
prc_rfc, rec_rfc

(0.8936985334719812, 0.8130553037171351)

In [113]:
print(classification_report(y_pred_rfc, y_test))

              precision    recall  f1-score   support

           1       0.96      0.85      0.90      5080
           2       0.18      0.38      0.25       376
           3       0.07      0.24      0.11        59

    accuracy                           0.81      5515
   macro avg       0.40      0.49      0.42      5515
weighted avg       0.89      0.81      0.85      5515



<font color=yellow>
5. Decision Tree
</font>

In [114]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)

X_train_dtc = sc.transform(X_train)
X_test_dtc = sc.transform(X_test)

In [115]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier(
                                    criterion='entropy',
                                    max_depth=100,
                                    random_state=42
)
dtc.fit(X_train_dtc, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=100, random_state=42)

In [116]:
y_pred_dtc = dtc.predict(X_test_dtc)
prc_dtc, rec_dtc = precision_score(y_pred_dtc, y_test, average='weighted'), recall_score(y_pred_dtc, y_test, average='weighted')
prc_dtc, rec_dtc

(0.7489314668408321, 0.7503173164097915)

In [117]:
print(classification_report(y_pred_dtc, y_test))

              precision    recall  f1-score   support

           1       0.86      0.87      0.86      4512
           2       0.26      0.25      0.26       803
           3       0.15      0.15      0.15       200

    accuracy                           0.75      5515
   macro avg       0.42      0.42      0.42      5515
weighted avg       0.75      0.75      0.75      5515



<font color=yellow>
6. Light GBM
</font>

In [118]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)

X_train_lgb = sc.transform(X_train)
X_test_lgb = sc.transform(X_test)

In [119]:
# 1. sklearn 패키지
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=400,
                                max_depth=100,
                                num_leaves=30,
                                objective='binary'
                                )
lgbm.fit(X_train_lgb, y_train)

LGBMClassifier(max_depth=100, n_estimators=400, num_leaves=30,
               objective='binary')

In [120]:
y_pred_lgb = lgbm.predict(X_test_lgb)
prc_lgb, rec_lgb = precision_score(y_pred_lgb, y_test, average='weighted'), recall_score(y_pred_lgb, y_test, average='weighted')
prc_lgb, rec_lgb

(0.9155195449767868, 0.8271985494106981)

In [121]:
print(classification_report(y_pred_lgb, y_test))

              precision    recall  f1-score   support

           1       0.97      0.86      0.91      5131
           2       0.19      0.45      0.27       329
           3       0.09      0.35      0.15        55

    accuracy                           0.83      5515
   macro avg       0.42      0.55      0.44      5515
weighted avg       0.92      0.83      0.86      5515



In [122]:
# 2. python 패키지
import lightgbm as lgb

params = {
    'n_estimators':400,
    'max_depth':100,
    'num_leaves':30,
    'objective':'binary',
    'metric':'binary_logloss',
    'min_data_in_leaf':1
}

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

num_round = 400

bst = lgb.train(params, train_data, num_round, test_data, verbose_eval=100, early_stopping_rounds=100)

[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 49631, number of negative: 0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 987
[LightGBM] [Info] Number of data points in the train set: 49631, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because the

In [123]:
y_pred_lgb2 = bst.predict(X_test)
y_pred_lgb2 = pd.Series(y_pred_lgb2).apply(lambda x: 1 if x>=0.5 else 0)
prc_lgb2, rec_lgb2 = precision_score(y_pred_lgb2, y_test, average='weighted'), recall_score(y_pred_lgb2, y_test, average='weighted')
prc_lgb2, rec_lgb2

(1.0, 0.8206708975521305)

In [124]:
print(classification_report(y_pred_lgb2, y_test))

              precision    recall  f1-score   support

           1       1.00      0.82      0.90      5515
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

    accuracy                           0.82      5515
   macro avg       0.33      0.27      0.30      5515
weighted avg       1.00      0.82      0.90      5515



<font color=yellow>
7. 앙상블
</font>

In [125]:
y_pred_rfc, y_pred_rfc.shape

(array([1, 1, 1, ..., 1, 1, 1], dtype=int64), (5515,))

In [126]:
y_pred_dtc, y_pred_dtc.shape

(array([1, 1, 1, ..., 2, 1, 1], dtype=int64), (5515,))

In [127]:
y_pred_lgb, y_pred_lgb.shape

(array([1, 1, 1, ..., 1, 1, 1], dtype=int64), (5515,))

In [128]:
y_pred_lgb2 = y_pred_lgb2.to_numpy()
y_pred_lgb2, y_pred_lgb2.shape

(array([1, 1, 1, ..., 1, 1, 1], dtype=int64), (5515,))

In [129]:
type(y_pred_rfc), type(y_pred_dtc), type(y_pred_lgb), type(y_pred_lgb2)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [130]:
pred_list = [y_pred_rfc, y_pred_dtc, y_pred_lgb, y_pred_lgb2]
y_pred_ensemble = []

for i in range(len(y_pred_rfc)):
    cnt = [0]*8
    
    for pred in pred_list:
        # print(pred[i])
        cnt[int(pred[i])] += 1
    y_pred_ensemble.append(cnt.index(max(cnt)))

In [131]:
prc_ensem, rec_ensem = precision_score(y_pred_ensemble, y_test, average='weighted'), recall_score(y_pred_ensemble, y_test, average='weighted')
prc_ensem, rec_ensem

(0.9599140504954119, 0.8264732547597462)

In [132]:
print(classification_report(y_pred_ensemble, y_test))

              precision    recall  f1-score   support

           1       0.99      0.84      0.91      5341
           2       0.09      0.49      0.16       150
           3       0.05      0.46      0.10        24

    accuracy                           0.83      5515
   macro avg       0.38      0.60      0.39      5515
weighted avg       0.96      0.83      0.88      5515



In [133]:
prc_rfc, rec_rfc, prc_dtc, rec_dtc, prc_lgb, rec_lgb, prc_lgb2, rec_lgb2, prc_ensem, rec_ensem

(0.8936985334719812,
 0.8130553037171351,
 0.7489314668408321,
 0.7503173164097915,
 0.9155195449767868,
 0.8271985494106981,
 1.0,
 0.8206708975521305,
 0.9599140504954119,
 0.8264732547597462)